In [405]:
import pandas as pd
import pickle
from statistics import mean
import numpy as np
from pathlib import Path

# Output from Dawid's notebook (CIF_feat dataframe)
# https://github.com/omdena/banking_unbanked/blob/feat-engineering/CIF_Clustering.ipynb
CIF_df = pd.read_csv('../data/CIF_feat.csv')
job_title = pd.read_excel('../data/JobTitle.xlsx',dtype={'CIF_NO':'Int64', 'job_title':str})

# Faster loading of original transaction dataset
if Path('../data/transaction.feather').is_file():
    transaction_df = pd.read_feather('../data/transaction.feather')
else:
    transaction_df = pd.read_excel('../data/TRANSACTION DATA OF BANK X_OCT DES 2019.xlsx')
    transaction_df.to_feather('../data/transaction.feather')
    
# Numbeo Scraped Dataset
with open('../data/region_dict.pickle', 'rb') as handle:
    region_living = pickle.load(handle)
with open('../data/area_dict.pickle', 'rb') as handle:
    area_living = pickle.load(handle)
    
# Indeed Scraped Dataset
with open('../data/scraped_jobs.pickle', 'rb') as handle:
    results_list = pickle.load(handle)

In [406]:
def remove_suffix(df_):
    for col in df_:
        df_[col] = df_[col].str.replace('(AREA|REGION)\s','')
    return df_

transaction_df[['REGION_ACCOUNT', 'AREA_ACCOUNT']] = remove_suffix(transaction_df[['REGION_ACCOUNT', 'AREA_ACCOUNT']])

C:\Users\Jun\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [407]:
# Get CIF_NO's Main account based on the highest transaction amount (Will be used for the Area/Region based salary)
CIF_main_acct = transaction_df.sort_values(['CIF_NO','AMT_TRX'], ascending=False).groupby('CIF_NO', as_index=False)['CIF_NO','AREA_ACCOUNT','AMT_TRX'].head(1)
CIF_df = CIF_df.merge(CIF_main_acct[['CIF_NO','AREA_ACCOUNT']], on='CIF_NO',how='left')

# Is_Religious based on voluntary religious contribution instead of obligatory zakat
CIF_df['Is_Religious'] = np.where(CIF_df['INFAQ monthly_amount'] != 0, 1, 0)
CIF_df['Is_Religious'].value_counts()

# Is_Management based on management key words
job_title['Is_Management'] = np.where(((job_title['Job Title'].str.contains('manager', case=False) == True)\
                                   | (job_title['Job Title'].str.contains('team leader', case=False) == True)\
                                   | (job_title['Job Title'].str.contains('head', case=False) == True))\
                                    , 1, 0)
job_title = job_title[~job_title.CIF_NO.isna()]

CIF_df = CIF_df.merge(job_title[['CIF_NO','Job Title','Is_Management']], on='CIF_NO',how='left')

## Consolidate Salary Data (Indeed.com > AREA > REGIONAL)

### Indeed Job Salary (Data Processing)

In [408]:
indeed_df = pd.DataFrame(results_list, columns=['QUERY','TITLE','COMPANY','LOCATION','SALARY']).drop_duplicates()
indeed_df = indeed_df[indeed_df.SALARY != '']
indeed_df.SALARY = indeed_df.SALARY.apply(lambda x: x.replace(' per bulan)','').replace(' ','').replace('Rp.','').replace('perbulan','').replace('.','').split('-'))
indeed_df['SALARY_MAX'] = indeed_df.SALARY.apply(lambda x: max([float(i) for i in x]))
indeed_df['SALARY_MIN'] = indeed_df.SALARY.apply(lambda x: min([float(i) for i in x]))
indeed_df['SALARY'] = indeed_df.SALARY.apply(lambda x: mean([float(i) for i in x]))

# # Ouput file for manual cleaning of job scraped
# results_df.to_csv('../data/temp_results_df.csv')

# Manually filtered
indeed_df = pd.read_csv('../data/temp_results_df.csv')[['QUERY','TITLE3','SALARY','SALARY_MAX','SALARY_MIN']]
indeed_df = indeed_df[indeed_df.TITLE3 != '0']
indeed_df = indeed_df[['TITLE3','SALARY','SALARY_MAX','SALARY_MIN']].rename(columns={'TITLE3':'Job Title'})
indeed_df = indeed_df.groupby('Job Title', as_index=False)['SALARY'].mean().sort_values('SALARY')
indeed_df

,Job Title,SALARY
0,Account Maintenance Staff,2.750000e+06
7,Consumer Administration Staff,3.375000e+06
8,Customer Service,3.492155e+06
10,Teller,4.233108e+06
9,General Support Staff,4.450000e+06
6,Business Banking Staff,4.668750e+06
2,Back Office,6.285000e+06
1,Area Manager,7.366667e+06
3,Branch Manager,7.562500e+06
4,Branch Operations & Service Manager,8.000000e+06


__Individual Income Tax__

Ref: https://www.indonesia-investments.com/finance/tax-system/item277

In [422]:
def deduct_income_tax(salary):
    if salary * 12 < 50000000:
        income_tax = 0.05
    elif salary * 12 < 250000000:
        income_tax = 0.15
    elif salary * 12 < 500000000:
        income_tax = 0.25
    else:
        income_tax = 0.30
    return salary * (1-income_tax)

In [424]:
indeed_df.SALARY = indeed_df.SALARY.apply(deduct_income_tax)

In [409]:
CIF_df = CIF_df.merge(indeed_df, on='Job Title', how='left')

### AREA/REGION Salary (Data Processing)
Ref: https://www.numbeo.com/property-investment/

Notebook: https://github.com/omdena/banking_unbanked/blob/master/code/ScrapeCostOfLiving.ipynb

In [411]:
def remove_suffix(df):
    for col in df:
        df[col] = df[col].str.replace('(AREA|REGION)\s','')
    return df
transaction_df[['REGION_ACCOUNT', 'AREA_ACCOUNT']] = remove_suffix(transaction_df[['REGION_ACCOUNT', 'AREA_ACCOUNT']])

region_area_map = transaction_df[['REGION_ACCOUNT', 'AREA_ACCOUNT']].drop_duplicates().sort_values('REGION_ACCOUNT')
region_area_map.head()

C:\Users\Jun\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,REGION_ACCOUNT,AREA_ACCOUNT
251,I/ MEDAN,ACEH
225,I/ MEDAN,MEDAN RAYA
51,I/ MEDAN,MEDAN KOTA
422,I/ MEDAN,BATAM
682,I/ MEDAN,PEMATANGSIANTAR


In [412]:
region_df = pd.DataFrame.from_dict(region_living).transpose().reset_index(drop=False).rename(columns={'index':'REGION_ACCOUNT'})
area_df = pd.DataFrame.from_dict(area_living).transpose().reset_index(drop=False).rename(columns={'index':'AREA_ACCOUNT'})

for dataframe in [region_df, area_df]:
    for col in dataframe:
        if 'ACCOUNT' not in col:
            dataframe[col] = dataframe[col].str[0].replace('?',np.nan).str.replace(',','').astype(float)

In [413]:
selected_cols = ['REGION_ACCOUNT',
                 'AREA_ACCOUNT',
                 'Average Monthly Net Salary (After Tax)_REGION',
                 'Average Monthly Net Salary (After Tax)_AREA',
                 'Apartment (3 bedrooms) in City Centre_REGION',
                 'Apartment (3 bedrooms) in City Centre_AREA',
                 'Apartment (3 bedrooms) Outside of Centre_REGION',
                 'Apartment (3 bedrooms) Outside of Centre_AREA']

renamed_cols = ['REGION_ACCOUNT',
                'AREA_ACCOUNT',
                'NET_SALARY_REGION',
                'NET_SALARY_AREA',
                'RENTAL_CENTRE_REGION',
                'RENTAL_CENTRE_AREA',
                'RENTAL_OUTSIDE_REGION',
                'RENTAL_OUTSIDE_AREA']

In [414]:
location_df = region_area_map.merge(region_df, on='REGION_ACCOUNT',how='left').merge(area_df, on='AREA_ACCOUNT', how='left', suffixes=('_REGION','_AREA'))
location_df = location_df[selected_cols]
location_df.columns = renamed_cols

location_df['NET_SALARY_AREA'].fillna(location_df['NET_SALARY_REGION'], inplace=True)

location_df['RENTAL_CENTRE_AREA'].fillna(location_df['RENTAL_CENTRE_REGION'], inplace=True)
location_df['RENTAL_OUTSIDE_AREA'].fillna(location_df['RENTAL_OUTSIDE_REGION'], inplace=True)

# Replace NA with interpolation done on Salary
location_df.set_index('NET_SALARY_AREA', inplace=True)
for col in location_df:
    if ('ACCOUNT' not in col) & (location_df[col].isna().sum() > 0):
        location_df[col] = location_df[col].interpolate(method='index').ffill().bfill()
location_df.reset_index(drop=False, inplace=True)

In [415]:
# Replace NA with interpolation done on Salary
location_df.set_index('NET_SALARY_AREA', inplace=True)
for col in location_df:
    if ('ACCOUNT' not in col) & (location_df[col].isna().sum() > 0):
        location_df[col] = location_df[col].interpolate(method='index').ffill().bfill()
location_df.reset_index(drop=False, inplace=True)

In [425]:
CIF_df = CIF_df.merge(location_df, on='AREA_ACCOUNT', how='left')
CIF_df[['CIF_NO','SALARY','NET_SALARY_AREA']].head()

,CIF_NO,SALARY,NET_SALARY_AREA
0,73000006,NaN,2998000.00
1,73000011,NaN,2998000.00
2,73000022,NaN,6478482.39
3,73000041,NaN,6478482.39
4,73000042,7.366667e+06,6478482.39
